# 📌 Model Converter Notebook

Questo notebook guida il processo di conversione del modello TensorFlow in TensorFlow Lite con l'aggiunta di metadati e compatibilità con l'API di Object Detection.

In [1]:
# Questa cella definisce una funzione analyze_saved_model(saved_model_dir) che analizza un modello TensorFlow salvato 
# e stampa dettagli strutturati relativi a:
#
# Firme disponibili: Mostra le firme disponibili nel modello.
# MetaGraph: Estrae e analizza la firma serving_default del modello.
# Tensori di input: Mostra i dettagli di nome, tipo, dimensione e forma dei tensori di input.
# Tensori di output: Mostra i dettagli di nome, tipo, dimensione e forma dei tensori di output.
# Firma serving_default: Stampa dettagli completi degli input e degli output strutturati della firma.
# In caso di errore, il codice cattura e stampa l'eccezione.

# Iniziamo importando le librerie necessarie
import tensorflow as tf
from tensorflow import lite
from tensorflow.python.tools import saved_model_utils
import os

# Definizione del percorso del modello salvato
SAVED_MODEL_PATH = "models/saved_model"

def analyze_saved_model(saved_model_dir):
    """ Analizza il modello salvato e stampa le informazioni dettagliate. """
    try:
        # Carica il modello salvato
        model = tf.saved_model.load(saved_model_dir)

        # Mostra le firme disponibili
        print("\n📌 **Firme disponibili nel modello:**")
        print("──────────────────────────────────")
        signatures = list(model.signatures.keys())
        for sig in signatures:
            print(f"🔹 {sig}")

        # Analizza il MetaGraph
        print("\n📌 **Analisi del MetaGraph**")
        print("──────────────────────────────────")
        meta_graph = saved_model_utils.get_meta_graph_def(saved_model_dir, tag_set="serve")
        signature_def = meta_graph.signature_def["serving_default"]

        # Analizza gli input
        print("\n📌 **Tensori di Input**")
        print("──────────────────────────────────")
        for key, tensor_info in signature_def.inputs.items():
            shape = "x".join(str(dim.size) if dim.size >= 0 else "?" for dim in tensor_info.tensor_shape.dim)
            print(f"🔹 Nome Tensore: {key}")
            print(f"   Nome Tecnico: {tensor_info.name}")
            print(f"   Tipo: {tf.dtypes.as_dtype(tensor_info.dtype).name}")
            print(f"   Dimensione: {shape}")
            print("   ───────────────────────────")

        # Analizza gli output
        print("\n📌 **Tensori di Output**")
        print("──────────────────────────────────")
        for key, tensor_info in signature_def.outputs.items():
            shape = "x".join(str(dim.size) if dim.size >= 0 else "?" for dim in tensor_info.tensor_shape.dim)
            print(f"🔹 Nome Tensore: {key}")
            print(f"   Nome Tecnico: {tensor_info.name}")
            print(f"   Tipo: {tf.dtypes.as_dtype(tensor_info.dtype).name}")
            print(f"   Dimensione: {shape}")
            print("   ───────────────────────────")

        # Analizza la firma "serving_default"
        print("\n📌 **Dettagli della firma 'serving_default'**")
        print("──────────────────────────────────")
        signature = model.signatures['serving_default']
        input_details = signature.structured_input_signature
        output_details = signature.structured_outputs

        print("\n📌 **Inputs della Firma**")
        print("──────────────────────────────────")
        for k, v in input_details[1].items():
            print(f"🔹 {k}: {v}")

        print("\n📌 **Outputs della Firma**")
        print("──────────────────────────────────")
        for k, v in output_details.items():
            print(f"🔹 {k}: {v}")

    except Exception as e:
        print(f"\n❌ Errore durante l'analisi del modello: {e}")

# Esegui l'analisi del modello
print("\n🚀 **Avvio dell'analisi del modello TensorFlow**")
analyze_saved_model(SAVED_MODEL_PATH)

print("\n✅ **Analisi completata con successo!** 🎉")



🚀 **Avvio dell'analisi del modello TensorFlow**

📌 **Firme disponibili nel modello:**
──────────────────────────────────
🔹 serving_default

📌 **Analisi del MetaGraph**
──────────────────────────────────

📌 **Tensori di Input**
──────────────────────────────────
🔹 Nome Tensore: images
   Nome Tecnico: serving_default_images:0
   Tipo: uint8
   Dimensione: 1x448x448x3
   ───────────────────────────

📌 **Tensori di Output**
──────────────────────────────────
🔹 Nome Tensore: raw_detection_scores
   Nome Tecnico: StatefulPartitionedCall:7
   Tipo: float32
   Dimensione: 1x12804x2
   ───────────────────────────
🔹 Nome Tensore: detection_multiclass_scores
   Nome Tecnico: StatefulPartitionedCall:3
   Tipo: float32
   Dimensione: 1x100x2
   ───────────────────────────
🔹 Nome Tensore: detection_scores
   Nome Tecnico: StatefulPartitionedCall:4
   Tipo: float32
   Dimensione: 1x100
   ───────────────────────────
🔹 Nome Tensore: detection_boxes
   Nome Tecnico: StatefulPartitionedCall:1
   Tipo:

In [2]:
# Questa cella definisce una nuova funzione di inferenza filtered_inference per rendere il modello compatibile con l’API 
# di Object Detection di TensorFlow. La funzione seleziona i 4 output richiesti dall’API tra gli 8 output disponibili 
# nel modello originale.
#
# - Input: Accetta lo stesso input dell'inferenza originale (images con forma (1, 448, 448, 3), tipo uint8).
# - Output: Restituisce un sottoinsieme degli output originali:
#   1. detection_boxes
#   2. detection_classes
#   3. detection_scores
#   4. num_detections
#
# Infine, il modello personalizzato viene salvato con una nuova firma di inferenza e, in caso di successo, 
# vengono stampati i dettagli dei tensori del nuovo modello.

# Percorso al modello originale SavedModel
SAVED_MODEL_PATH = "models/saved_model"

# Percorso per salvare il nuovo modello
NEW_SAVED_MODEL_PATH = "models/custom_model"

# Caricamento del modello originale
print(f"Caricamento del modello originale da: {SAVED_MODEL_PATH}")
model = tf.saved_model.load(SAVED_MODEL_PATH)

# Estrai la signature originale
original_infer = model.signatures['serving_default']

# Definizione della nuova funzione di inferenza con output in formato flat list
@tf.function(input_signature=[tf.TensorSpec(shape=(1, 448, 448, 3), dtype=tf.uint8, name='images')])
def filtered_inference(*args, **kwargs):
    # Esegui l'inferenza originale
    original_outputs = original_infer(*args, **kwargs)
    # Restituisci gli output
    return {
        'detection_boxes': original_outputs['detection_boxes'],
        'detection_classes': original_outputs['detection_classes'],
        'detection_scores': original_outputs['detection_scores'],
        'num_detections': original_outputs['num_detections']
    }

# Salvataggio del nuovo modello con la signature personalizzata
print(f"Salvataggio del nuovo modello con signature personalizzata in: {NEW_SAVED_MODEL_PATH}")
tf.saved_model.save(
    model,
    NEW_SAVED_MODEL_PATH,
    signatures={'filtered_inference': filtered_inference.get_concrete_function()}
)
print(f"Nuovo modello salvato in: {NEW_SAVED_MODEL_PATH}")

# Verifica del nuovo modello: Stampa dei dettagli dei tensori
print("\n📌 Verifica del nuovo modello salvato")
try:
    # Carica il nuovo modello salvato
    new_model = tf.saved_model.load(NEW_SAVED_MODEL_PATH)
    
    # Accedi alla nuova firma 'filtered_inference'
    new_signature = new_model.signatures['filtered_inference']
    
    # Recupera i dettagli di input e output della firma
    new_input_details = new_signature.structured_input_signature
    new_output_details = new_signature.structured_outputs

    # Stampa i dettagli degli input
    print("\n📌 Dettagli della firma 'filtered_inference'")
    print("== Inputs ==")
    for key, value in new_input_details[1].items():
        print(f"🔹 {key}: {value}")
    
    # Stampa i dettagli degli output
    print("\n== Outputs ==")
    for key, value in new_output_details.items():
        print(f"🔹 {key}: {value}")

except Exception as e:
    print(f"\n❌ Errore durante la verifica del nuovo modello: {e}")


Caricamento del modello originale da: models/saved_model
Salvataggio del nuovo modello con signature personalizzata in: models/custom_model
INFO:tensorflow:Assets written to: models/custom_model\assets


INFO:tensorflow:Assets written to: models/custom_model\assets


Nuovo modello salvato in: models/custom_model

📌 Verifica del nuovo modello salvato

📌 Dettagli della firma 'filtered_inference'
== Inputs ==
🔹 images: TensorSpec(shape=(1, 448, 448, 3), dtype=tf.uint8, name='images')

== Outputs ==
🔹 detection_scores: TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_scores')
🔹 detection_classes: TensorSpec(shape=(1, 100), dtype=tf.float32, name='detection_classes')
🔹 detection_boxes: TensorSpec(shape=(1, 100, 4), dtype=tf.float32, name='detection_boxes')
🔹 num_detections: TensorSpec(shape=(1,), dtype=tf.float32, name='num_detections')


In [16]:
# Conversione del modello SavedModel in TFLite con diverse tecniche di quantizzazione
# Questa cella converte il modello SavedModel in tre versioni:
# - Senza quantizzazione
# - Con quantizzazione dinamica
# - Con quantizzazione float16

# Percorsi dei modelli
SAVED_MODEL_PATH = "models/custom_model"
TFLITE_MODEL_PATH = "models/custom_model.tflite"
TFLITE_QUANT_MODEL_PATH = "models/custom_model_quantized.tflite"
TFLITE_FLOAT16_MODEL_PATH = "models/custom_model_float16.tflite"

# Funzione per convertire il modello in TFLite
def convert_to_tflite(saved_model_path, tflite_model_path, quantization_type=None):
    """
    Converte un modello SavedModel in TFLite.

    Args:
        saved_model_path (str): Percorso del modello SavedModel.
        tflite_model_path (str): Percorso dove salvare il modello TFLite.
        quantization_type (str, opzionale): Tipo di quantizzazione ("dynamic", "float16" o None).
    """
    converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
    
    if quantization_type == "dynamic":
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
    elif quantization_type == "float16":
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]
    
    tflite_model = converter.convert()
    with open(tflite_model_path, 'wb') as f:
        f.write(tflite_model)
    print(f"✅ Modello TFLite salvato in: {tflite_model_path}")

# Conversione del modello in TFLite senza quantizzazione
print("🔄 Conversione del modello SENZA quantizzazione...")
convert_to_tflite(SAVED_MODEL_PATH, TFLITE_MODEL_PATH, quantization_type=None)

# Conversione del modello in TFLite con quantizzazione dinamica
print("\n🔄 Conversione del modello CON quantizzazione dinamica...")
convert_to_tflite(SAVED_MODEL_PATH, TFLITE_QUANT_MODEL_PATH, quantization_type="dynamic")

# Conversione del modello in TFLite con quantizzazione float16
print("\n🔄 Conversione del modello CON quantizzazione float16...")
convert_to_tflite(SAVED_MODEL_PATH, TFLITE_FLOAT16_MODEL_PATH, quantization_type="float16")

# Funzione per calcolare la dimensione del file
def get_model_size(file_path):
    size_in_kb = os.path.getsize(file_path) / 1024
    return size_in_kb

# Stampa delle dimensioni dei modelli
print("\n📏 Dimensioni dei modelli:")
size_no_quant = get_model_size(TFLITE_MODEL_PATH)
size_quant_dynamic = get_model_size(TFLITE_QUANT_MODEL_PATH)
size_float16 = get_model_size(TFLITE_FLOAT16_MODEL_PATH)

print(f"🚀 Modello SENZA quantizzazione: {size_no_quant:.2f} KB")
print(f"⚡ Modello CON quantizzazione dinamica: {size_quant_dynamic:.2f} KB")
print(f"🔍 Modello CON quantizzazione float16: {size_float16:.2f} KB")

print("\n✅ Conversione completata per tutti i modelli.")


🔄 Conversione del modello SENZA quantizzazione...
✅ Modello TFLite salvato in: models/custom_model.tflite

🔄 Conversione del modello CON quantizzazione dinamica...
✅ Modello TFLite salvato in: models/custom_model_quantized.tflite

🔄 Conversione del modello CON quantizzazione float16...
✅ Modello TFLite salvato in: models/custom_model_float16.tflite

📏 Dimensioni dei modelli:
🚀 Modello SENZA quantizzazione: 11116.20 KB
⚡ Modello CON quantizzazione dinamica: 2980.32 KB
🔍 Modello CON quantizzazione float16: 5654.55 KB

✅ Conversione completata per tutti i modelli.


In [23]:
# Scrittura e Analisi dei Metadati del Modello TFLite
# Questa cella utilizza la Metadata Writer API di TensorFlow Lite per:
# 1. Aggiungere i metadati necessari al modello non quantizzato per supportare l'API Object Detection di Android.
# 2. Analizzare e visualizzare in modo chiaro i metadati aggiunti al modello.

# ========================
# Parte 1: Scrittura dei Metadati
# ========================

from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils

# Definizione dei percorsi e parametri per il modello e i metadati
ObjectDetectorWriter = object_detector.MetadataWriter
_MODEL_PATH = "models/custom_model.tflite"  # Modello TFLite senza quantizzazione
_LABEL_FILE = "models/label_map.pbtxt"          # File delle etichette per il modello
_SAVE_TO_PATH = "models/plate_recognizer.tflite"  # Modello finale con metadati
_INPUT_NORM_MEAN = 127.5                   # Parametro per la normalizzazione degli input
_INPUT_NORM_STD = 127.5                    # Parametro per la normalizzazione degli input

# Creazione del Metadata Writer per il modello
writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(_MODEL_PATH), 
    [_INPUT_NORM_MEAN], 
    [_INPUT_NORM_STD],
    [_LABEL_FILE]
)

# Popolazione dei metadati nel modello e salvataggio del modello finale
writer_utils.save_file(writer.populate(), _SAVE_TO_PATH)
print(f"✅ Modello con metadati salvato in: {_SAVE_TO_PATH}")

# ========================
# Parte 2: Analisi e Visualizzazione dei Metadati
# ========================

import json
from tflite_support import metadata
from IPython.display import display, HTML

# Percorso del modello finale con metadati
model_path = _SAVE_TO_PATH

# Funzione per analizzare e stampare i metadati del modello in modo schematizzato e leggibile
def display_model_metadata(model_path):
    """Analizza i metadati del modello TensorFlow Lite e li stampa in modo leggibile.
    
    Args:
        model_path (str): Percorso al file del modello TFLite.
    """
    try:
        # Creazione di un MetadataDisplayer per il modello specificato
        displayer = metadata.MetadataDisplayer.with_model_file(model_path)
        
        # Estrazione dei metadati in formato JSON
        metadata_json = displayer.get_metadata_json()
        metadata_dict = json.loads(metadata_json)
        
        # Funzione per generare HTML colorato
        def format_html_section(title, content, color="blue"):
            return f"""
            <div style="border: 1px solid {color}; border-radius: 5px; padding: 10px; margin-bottom: 10px;">
                <h3 style="color: {color};">{title}</h3>
                <pre style="font-size: 14px;">{content}</pre>
            </div>
            """
        
        # Stampa del titolo
        html_content = f"<h2 style='color: green;'>📌 Analisi dei Metadati del Modello</h2>"
        
        # Dettagli generali del modello
        html_content += format_html_section("📄 Dettagli Generali",
                                             json.dumps(metadata_dict.get("name", "N/A"), indent=4))
        
        # Input tensor metadata
        input_metadata = metadata_dict["subgraph_metadata"][0]["input_tensor_metadata"][0]
        html_content += format_html_section("📥 Input Tensor Metadata",
                                             json.dumps(input_metadata, indent=4), color="purple")
        
        # Output tensor metadata
        for output_tensor in metadata_dict["subgraph_metadata"][0]["output_tensor_metadata"]:
            html_content += format_html_section(f"📤 Output Tensor Metadata: {output_tensor['name']}",
                                                 json.dumps(output_tensor, indent=4), color="orange")
        
        # File associati
        associated_files = displayer.get_packed_associated_file_list()
        associated_files_section = "\n".join([f"📎 {file}" for file in associated_files]) if associated_files else "❌ Nessun file associato trovato."
        html_content += format_html_section("📂 File Associati", associated_files_section, color="red")
        
        # Mostra il risultato come HTML
        display(HTML(html_content))
    
    except Exception as e:
        print(f"❌ Si è verificato un errore durante l'analisi dei metadati: {e}")

# Analizza e stampa i metadati del modello finale con metadati
display_model_metadata(model_path)


✅ Modello con metadati salvato in: models/plate_recognizer.tflite


In [24]:
# Copyright 2020 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Writes metadata and label file to the object detection models.

Lo script originale fornito da TensorFlow è stato modificato per mappare correttamente
i metadati ai tensori di output nell'ordine corretto.

Inoltre, lo script è stato adattato per scrivere i metadati su tre modelli:
    - Modello senza quantizzazione
    - Modello con quantizzazione dinamica
    - Modello con quantizzazione Float16
"""

from __future__ import absolute_import, division, print_function

import flatbuffers
from tflite_support import metadata_schema_py_generated as _metadata_fb
from tflite_support import metadata as _metadata

# File paths per i tre modelli convertiti
MODELS = {
    "non_quantized": "models/custom_model.tflite",
    "quantized_dynamic": "models/custom_model_quantized.tflite",
    "quantized_float16": "models/custom_model_float16.tflite",
}

LABEL_FILE = "models/label_map.pbtxt"

class ModelSpecificInfo:
    """Holds information for an object detector."""
    def __init__(self, name, version, image_width, image_height, mean, std):
        self.name = name
        self.version = version
        self.image_width = image_width
        self.image_height = image_height
        self.mean = mean
        self.std = std

MODEL_INFO = ModelSpecificInfo(
    name="PlateRecognizer",
    version="3.0",
    image_width=300,
    image_height=300,
    mean=[127.5],
    std=[127.5])

class MetadataPopulatorForObjectDetector:
    """Populates the metadata for an object detection model."""

    def __init__(self, model_file, export_model_file, model_info, label_file_path):
        self.model_file = model_file
        self.export_model_file = export_model_file
        self.model_info = model_info
        self.label_file_path = label_file_path
        self.metadata_buf = None

    def populate(self):
        """Creates metadata and then populates it for an object detection model."""
        self._create_metadata()
        self._populate_metadata()

    def _create_metadata(self):
        """Creates the metadata for an object detection model."""

        # Model metadata
        model_meta = _metadata_fb.ModelMetadataT()
        model_meta.name = self.model_info.name
        model_meta.description = (
            "Identify objects in an image and provide their locations."
        )
        model_meta.version = self.model_info.version
        model_meta.license = ("Apache License. Version 2.0 "
                              "http://www.apache.org/licenses/LICENSE-2.0.")

        # Input tensor metadata
        input_meta = _metadata_fb.TensorMetadataT()
        input_meta.name = "image"
        input_meta.description = "Input image to be detected."
        input_meta.content = _metadata_fb.ContentT()
        input_meta.content.contentProperties = _metadata_fb.ImagePropertiesT()
        input_meta.content.contentProperties.colorSpace = _metadata_fb.ColorSpaceType.RGB
        input_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.ImageProperties
        
        input_normalization = _metadata_fb.ProcessUnitT()
        input_normalization.optionsType = _metadata_fb.ProcessUnitOptions.NormalizationOptions
        input_normalization.options = _metadata_fb.NormalizationOptionsT()
        input_normalization.options.mean = self.model_info.mean
        input_normalization.options.std = self.model_info.std
        input_meta.processUnits = [input_normalization]

        input_stats = _metadata_fb.StatsT()
        input_stats.max = [255.0]
        input_stats.min = [0.0]
        input_meta.stats = input_stats

        # Output tensors metadata
        output_tensors = []

        # Number of detections
        num_detections_meta = _metadata_fb.TensorMetadataT()
        num_detections_meta.name = "number of detections"
        num_detections_meta.description = "The number of detected plates."
        num_detections_meta.content = _metadata_fb.ContentT()
        num_detections_meta.content.contentProperties = _metadata_fb.FeaturePropertiesT()
        num_detections_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.FeatureProperties
        output_tensors.append(num_detections_meta)

        # Location (bounding boxes)
        location_meta = _metadata_fb.TensorMetadataT()
        location_meta.name = "location"
        location_meta.description = "The locations of the detected plates."
        location_meta.content = _metadata_fb.ContentT()
        location_meta.content.contentProperties = _metadata_fb.BoundingBoxPropertiesT()
        location_meta.content.contentProperties.index = [1, 0, 3, 2]
        location_meta.content.contentProperties.type = _metadata_fb.BoundingBoxType.BOUNDARIES
        location_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.BoundingBoxProperties
        output_tensors.append(location_meta)

        # Category
        category_meta = _metadata_fb.TensorMetadataT()
        category_meta.name = "category"
        category_meta.description = "The categories of the detected plates."
        category_meta.content = _metadata_fb.ContentT()
        category_meta.content.contentProperties = _metadata_fb.FeaturePropertiesT()
        category_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.FeatureProperties
        output_tensors.append(category_meta)

        # Score
        score_meta = _metadata_fb.TensorMetadataT()
        score_meta.name = "score"
        score_meta.description = "The confidence scores of the detected plates."
        score_meta.content = _metadata_fb.ContentT()
        score_meta.content.contentProperties = _metadata_fb.FeaturePropertiesT()
        score_meta.content.contentPropertiesType = _metadata_fb.ContentProperties.FeatureProperties
        output_tensors.append(score_meta)

        # Associated label file
        label_file = _metadata_fb.AssociatedFileT()
        label_file.name = os.path.basename(self.label_file_path)
        label_file.description = "Labels for categories that the model can recognize."
        label_file.type = _metadata_fb.AssociatedFileType.TENSOR_VALUE_LABELS
        category_meta.associatedFiles = [label_file]

        # Subgraph metadata
        subgraph = _metadata_fb.SubGraphMetadataT()
        subgraph.inputTensorMetadata = [input_meta]
        subgraph.outputTensorMetadata = output_tensors
        model_meta.subgraphMetadata = [subgraph]

        # Generate metadata buffer
        b = flatbuffers.Builder(0)
        b.Finish(
            model_meta.Pack(b),
            _metadata.MetadataPopulator.METADATA_FILE_IDENTIFIER)
        self.metadata_buf = b.Output()

    def _populate_metadata(self):
        """Populates metadata and label file to the model file."""
        populator = _metadata.MetadataPopulator.with_model_file(self.export_model_file)
        populator.load_metadata_buffer(self.metadata_buf)
        populator.load_associated_files([self.label_file_path])
        populator.populate()

def main():
    for model_name, model_path in MODELS.items():
        export_model_path = model_path.replace(".tflite", "_with_metadata.tflite")

        # Copia il modello originale
        tf.io.gfile.copy(model_path, export_model_path, overwrite=True)

        # Popolazione dei metadati
        populator = MetadataPopulatorForObjectDetector(
            model_path, export_model_path, MODEL_INFO, LABEL_FILE)
        populator.populate()

        # Generazione file JSON con i metadati
        displayer = _metadata.MetadataDisplayer.with_model_file(export_model_path)
        export_json_file = export_model_path.replace(".tflite", ".json")
        with open(export_json_file, "w") as f:
            f.write(displayer.get_metadata_json())

        print(f"✅ Metadati aggiunti a {export_model_path}")
        print(f"📂 Metadati salvati in: {export_json_file}")

if __name__ == "__main__":
    main()


✅ Metadati aggiunti a models/custom_model_with_metadata.tflite
📂 Metadati salvati in: models/custom_model_with_metadata.json
✅ Metadati aggiunti a models/custom_model_quantized_with_metadata.tflite
📂 Metadati salvati in: models/custom_model_quantized_with_metadata.json
✅ Metadati aggiunti a models/custom_model_float16_with_metadata.tflite
📂 Metadati salvati in: models/custom_model_float16_with_metadata.json
